In [1]:
%cd ..

g:\My Drive\Professional_Backups\Professional\Erdós\AP-Outcomes-to-University-Metrics


In [2]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import data_loaders
import distance_dorm_descriptors as ddd

## Pull and process Georgia outcomes

In [ ]:
url_2019='https://download.gosa.ga.gov/2019/AP_2019_FEB_24_2020.csv'
url_2020='https://download.gosa.ga.gov/2020/AP_2020_JUN_21_2021.csv'
url_2021='https://download.gosa.ga.gov/2021/AP_2021_Dec062021.csv'
url_2022='https://download.gosa.ga.gov/2022/AP_2022_Apr102023.csv'
url_2023='https://download.gosa.ga.gov/2023/AP_2022-23_2024-04-02_14_14_37.csv'

ga_2019=pd.read_csv(url_2019)
ga_2020=pd.read_csv(url_2020)
ga_2021=pd.read_csv(url_2021)
ga_2022=pd.read_csv(url_2022)
ga_2023=pd.read_csv(url_2023)

# Year numbering modified to be consistent to other states as starting year not year of exam

ga_2019['Year']=2018
ga_2020['Year']=2019
ga_2021['Year']=2020
ga_2022['Year']=2021
ga_2023['Year']=2022


ga_2023=ga_2023.rename(columns={'NUMBER_TESTS_3_OR_HIGHER':'NOTESTS_3ORHIGHER'})
ga_2023=ga_2023[['LONG_SCHOOL_YEAR', 'SCHOOL_DISTRCT_CD', 'SCHOOL_DSTRCT_NM',
       'INSTN_NUMBER', 'INSTN_NAME', 'TEST_CMPNT_TYP_NM',
       'NUMBER_OF_STUDENTS_TESTED', 'NUMBER_TESTS_TAKEN', 'NOTESTS_3ORHIGHER',
       'Year']] # Naming convention fix

years=[ga_2019,ga_2020,ga_2021,ga_2022,ga_2023]
ga_total=pd.concat(years)

### kept and new information GA

Kept: ['SCHOOL_DSTRCT_NM', 'INSTN_NUMBER', 'TEST_CMPNT_TYP_NM','NUMBER_TESTS_TAKEN', 'NOTESTS_3ORHIGHER']
New: ['Year', 'County']

All non-county data dropped except Atlanta Public Schools which has been as part of Fulton county

Data that has 'TRS' was dropped

School specific data is dropped

In [169]:
kept_keys=['SCHOOL_DSTRCT_NM', 'TEST_CMPNT_TYP_NM','NUMBER_TESTS_TAKEN', 'NOTESTS_3ORHIGHER','Year']
ga_partial=ga_total.loc[ga_total['INSTN_NUMBER'].isin(['SCHOOL_ALL','ALL'])==True][kept_keys]

In [170]:
# Missing data fixer
ga_partial=ga_partial.dropna()
ga_partial=ga_partial[ga_partial['NOTESTS_3ORHIGHER'].str.contains('TFS',na=False)==False]
ga_partial=ga_partial[ga_partial['NUMBER_TESTS_TAKEN'].str.contains('TFS',na=False)==False]
ga_partial=ga_partial.astype({'NUMBER_TESTS_TAKEN':'float','NOTESTS_3ORHIGHER':'float'})

In [171]:
# Atlanta fixer
temp1=ga_partial[ga_partial['SCHOOL_DSTRCT_NM'].str.contains('Atlanta')]
temp2=ga_partial[ga_partial['SCHOOL_DSTRCT_NM'].str.contains('Fulton')]
temp3=pd.concat([temp1,temp2]).groupby(['Year','TEST_CMPNT_TYP_NM']).sum().reset_index()
temp3['SCHOOL_DSTRCT_NM']='Fulton County'
ga_partial=ga_partial[ga_partial['SCHOOL_DSTRCT_NM']!='Fulton County']
ga_partial=pd.concat([ga_partial,temp3])

In [172]:
ga_partial.tail()

,SCHOOL_DSTRCT_NM,TEST_CMPNT_TYP_NM,NUMBER_TESTS_TAKEN,NOTESTS_3ORHIGHER,Year
175,Fulton County,Spanish Language,277.0,250.0,2022
176,Fulton County,Spanish Literature,15.0,13.0,2022
177,Fulton County,Statistics,1169.0,837.0,2022
178,Fulton County,U.S. History,1896.0,895.0,2022
179,Fulton County,World History,2755.0,1903.0,2022


In [173]:
# Pass rates 
ga_partial['Pass Rate']=ga_partial['NOTESTS_3ORHIGHER']/ga_partial['NUMBER_TESTS_TAKEN']

# Counties
ga_partial=ga_partial[ga_partial['SCHOOL_DSTRCT_NM'].str.contains('County')]
ga_partial['County']=ga_partial['SCHOOL_DSTRCT_NM'].str.split(' County',n=1,expand=True)[0]

ga_partial.to_pickle('data/GA_pickled/GA_2018-22_outcomes.pkl')

In [174]:
carnegie=data_loaders.gimmeCarnegieLimtedWithLocations()
ga_outcomes=ga_partial
county_income=data_loaders.gimmeCountyIncomes()
county_pop= pd.read_csv('data/uscounties.csv')

In [9]:
ga_pop=county_pop.loc[county_pop['state_id']=='GA'].reset_index()[['county','lat','lng','population']]
ga_income=county_income.loc[county_income['State_Abbreviation']=='GA'].reset_index()

In [10]:
# Testing geopy
coord1 = (carnegie.iloc[1].latitude,carnegie.iloc[1].longitude)
coord2 = (ga_pop.iloc[1].lat,ga_pop.iloc[1].lng)
print(distance(coord1,coord2).miles)

163.27496179983422


These district are double counties and excluded

In [175]:
exceptions=ga_outcomes[ga_outcomes['County'].isin(county_pop['county'].values)==False]['County'].drop_duplicates()
print(exceptions)

679     Savannah-Chatham
5941    Griffin-Spalding
6173     Thomaston-Upson
Name: County, dtype: object


In [176]:
ga_outcomes_no_exceptions=ga_outcomes[ga_outcomes['County'].isin(county_pop['county'].values)==True]

In [177]:
ga_outcomes_no_exceptions=ga_outcomes_no_exceptions.loc[ga_outcomes_no_exceptions['TEST_CMPNT_TYP_NM']=='ALL Subjects']

In [21]:
carnegie_full = pd.read_excel('data/CCIHE2021-PublicData.xlsx',sheet_name='Data')
carnegie['stem_rsd'] = carnegie_full['stem_rsd']
carnegie['anenr1920'] = carnegie_full['anenr1920']

In [22]:
ga_pop['closest_five_r1r2_avg']=ga_pop.apply(lambda x: ddd.closest_five(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest_five_public_avg']=ga_pop.apply(lambda x: ddd.closest_five(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_private_nfp_avg']=ga_pop.apply(lambda x: ddd.closest_five(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest_five_landgrant_avg']=ga_pop.apply(lambda x: ddd.closest_five(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_stem_avg'] = ga_pop.apply(lambda x: ddd.closest_five(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [23]:
ga_pop['closest_five_avg_enrollment_r1r2']=ga_pop.apply(lambda x: ddd.closest_five_enrollment(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_public']=ga_pop.apply(lambda x: ddd.closest_five_enrollment(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_private_nfp']=ga_pop.apply(lambda x: ddd.closest_five_enrollment(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_landgrant']=ga_pop.apply(lambda x: ddd.closest_five_enrollment(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_stem'] = ga_pop.apply(lambda x: ddd.closest_five_enrollment(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [24]:
ga_pop['closest_five_avg_dormrooms_r1r2']=ga_pop.apply(lambda x: ddd.closest_five_rooms(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_public']=ga_pop.apply(lambda x: ddd.closest_five_rooms(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_private_nfp']=ga_pop.apply(lambda x: ddd.closest_five_rooms(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_landgrant']=ga_pop.apply(lambda x: ddd.closest_five_rooms(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_stem'] = ga_pop.apply(lambda x: ddd.closest_five_rooms(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [83]:
ga_income_combiner=ga_income.rename(columns={'County':'county'})[['county','2018','2019','2020','2021','2022']]
ga_income_combiner=ga_income_combiner.melt(id_vars=['county'],value_vars=['2018','2019','2020','2021','2022'],var_name='Year',value_name='income')

In [84]:
ga_data=pd.merge(ga_pop,ga_income_combiner,on='county')
ga_data.head()

,county,lat,lng,population,closest_five_r1r2_avg,closest_five_public_avg,closest_five_private_nfp_avg,closest_five_landgrant_avg,closest_five_stem_avg,closest_five_avg_enrollment_r1r2,...,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrant,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrant,closest_five_avg_dormrooms_stem,Year,income
0,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2018,87609
1,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2019,93307
2,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2020,92999
3,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2021,101688
4,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2022,100614


In [86]:
ga_data.Year=ga_data.Year.astype('int')

In [178]:
ga_outcomes_fin=pd.merge(ga_outcomes_no_exceptions.rename(columns={'County':'county'}),ga_data,on=['county','Year'],validate='one_to_many')

In [179]:
ga_outcomes_fin.tail()

,SCHOOL_DSTRCT_NM,TEST_CMPNT_TYP_NM,NUMBER_TESTS_TAKEN,NOTESTS_3ORHIGHER,Year,Pass Rate,county,lat,lng,population,...,closest_five_avg_enrollment_public,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrant,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrant,closest_five_avg_dormrooms_stem,income
531,Fulton County,ALL Subjects,23510.0,15234.0,2018,0.647980,Fulton,33.7902,-84.467,1061944,...,23673.2,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,87609
532,Fulton County,ALL Subjects,21627.0,14897.0,2019,0.688815,Fulton,33.7902,-84.467,1061944,...,23673.2,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,93307
533,Fulton County,ALL Subjects,19252.0,11841.0,2020,0.615053,Fulton,33.7902,-84.467,1061944,...,23673.2,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,92999
534,Fulton County,ALL Subjects,21857.0,13908.0,2021,0.636318,Fulton,33.7902,-84.467,1061944,...,23673.2,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,101688
535,Fulton County,ALL Subjects,24411.0,16278.0,2022,0.666831,Fulton,33.7902,-84.467,1061944,...,23673.2,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,100614


In [180]:
#COUNTY,STUDENTS_TESTED,EXAM_COUNT,EXAMS_3_OR_ABOVE,PERCENT_3_OR_ABOVE,Year,Latitude,Longitude,population,per_capita_income,closest_five_r1r2_avg,closest_five_public_avg,closest_five_private_nfp_avg,closest_five_landgrnt_avg,closest_five_stem_avg,closest_five_avg_enrollment_r1r2,closest_five_avg_enrollment_public,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrnt,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrant,closest_five_avg_dormrooms_stem
#['SCHOOL_DSTRCT_NM', 'TEST_CMPNT_TYP_NM', 'NUMBER_TESTS_TAKEN','NOTESTS_3ORHIGHER', 'Year', 'Pass Rate', 'county', 'lat', 'lng','population', 'closest_five_r1r2_avg', 'closest_five_public_avg','closest_five_private_nfp_avg', 'closest_five_landgrnt_avg','closest_five_stem_avg', 'closest_five_avg_enrollment_r1r2','closest_five_avg_enrollment_public','closest_five_avg_enrollment_private_nfp','closest_five_avg_enrollment_landgrnt','closest_five_avg_enrollment_stem', 'closest_five_avg_dormrooms_r1r2','closest_five_avg_dormrooms_public','closest_five_avg_dormrooms_private_nfp','closest_five_avg_dormrooms_landgrnt','closest_five_avg_dormrooms_stem', 'year', 'income']
ga_outcomes_fin=ga_outcomes_fin.rename(columns={'NUMBER_TESTS_TAKEN':'EXAM_COUNT','NOTESTS_3ORHIGHER':'EXAMS_3_OR_ABOVE','lat':'Latitude','lng':'Longitude','income':'per_capita_income','county':'COUNTY','Pass Rate':'PERCENT_3_OR_ABOVE'})

In [181]:
ga_outcomes_fin.eval('PERCENT_3_OR_ABOVE=PERCENT_3_OR_ABOVE*100',inplace=True)


In [182]:
ga_outcomes_fin.head()

,SCHOOL_DSTRCT_NM,TEST_CMPNT_TYP_NM,EXAM_COUNT,EXAMS_3_OR_ABOVE,Year,PERCENT_3_OR_ABOVE,COUNTY,Latitude,Longitude,population,...,closest_five_avg_enrollment_public,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrant,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrant,closest_five_avg_dormrooms_stem,per_capita_income
0,Appling County,ALL Subjects,92.0,37.0,2018,40.217391,Appling,31.7492,-82.2889,18441,...,9106.8,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,33662
1,Atkinson County,ALL Subjects,37.0,8.0,2018,21.621622,Atkinson,31.2971,-82.8801,8265,...,6876.4,2474.4,21696.0,27405.6,4389.8,912.2,628.4,4156.2,4772.0,29212
2,Baldwin County,ALL Subjects,46.0,13.0,2018,28.260870,Baldwin,33.0693,-83.2495,43778,...,9545.2,2734.2,22071.6,23945.4,4904.2,909.0,900.0,5065.4,4904.2,33125
3,Banks County,ALL Subjects,37.0,16.0,2018,43.243243,Banks,34.3543,-83.4972,18381,...,16641.6,2717.4,21914.8,34447.6,7334.6,2599.6,622.4,5643.6,7334.6,37740
4,Barrow County,ALL Subjects,757.0,355.0,2018,46.895641,Barrow,33.9932,-83.7127,84399,...,16912.8,1307.0,27932.4,29515.6,6238.8,2204.0,280.4,6317.4,6238.8,37059


In [183]:
from sklearn.model_selection import train_test_split
training, testing = train_test_split(ga_outcomes_fin, test_size = 0.2, random_state = 226)

In [184]:
training.to_pickle('data/GA_pickled/training.pkl')
testing.to_pickle('data/GA_pickled/testing.pkl')

In [185]:
ga_outcomes_fin.to_pickle('data/GA_pickled/ga_outcomes_full_data.pkl')